In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [3]:
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'
DISTANCE = 10
ROUNDS = 10
LOGICAL = 0
_RESETS = False

md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md['meas_level'] == 1]
md = md[md['distance']==DISTANCE]
md = md[md['rounds']==ROUNDS]
md = md[md['logical']==LOGICAL]

md = md[md['notebook_name']=='mean_pos_sherbrooke']

md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets,num_qubit,double_msmt,xbasis,path_info
2667,2024-03-11 12:51:17.882000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf0d9v78p0008mdt3g,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2666,2024-03-11 12:51:11.681000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf0bsr43fg008x5td0,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2665,2024-03-11 12:51:05.921000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf0a9v78p0008mdt30,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2664,2024-03-11 12:50:59.625000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf08s4td0g008cqmk0,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2663,2024-03-11 12:50:53.619000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf078267h0008gdks0,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2662,2024-03-11 12:50:47.103000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf05gv78p0008mdt2g,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2661,2024-03-11 12:50:35.810000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf02r417q0008ytz9g,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2660,2024-03-11 12:50:29.775000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqf018v78p0008mdt20,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2659,2024-03-11 12:50:22.473000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqezzq417q0008ytz8g,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN
2658,2024-03-11 12:50:16.452000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqqezy7r43fg008x5tc0,"[1*1e6/d*T, RepCode Mean Positions]",1.0,111.0,NaN,0.0005,NaN,...,0,NaN,Mean positions,NaN,NaN,False,NaN,NaN,NaN,NaN


# Initialize the decoder

In [4]:
import numpy as np
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict
from Scratch import load_calibration_memory
import cpp_soft_info

rel_error = 1
lin = [0.4, 0.7, 20]
num_points = 51
# lin = [0.4, 0.7, 1]
# num_points = 1
bandwidths = np.linspace(lin[0], lin[1], lin[2])

noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = md.iloc[0]['execution_date'])

# Get the KDEs
all_memories = load_calibration_memory(provider, md.iloc[0]["job_id"])
kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=1, absError=-1, num_points=num_points)

Specified job execution date: 2024-03-11 13:49:38.212792+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-11 10:59:11.387273+00:00.


In [23]:
from soft_info import get_repcode_IQ_map, counts_to_det_syndr
import stim
import pymatching
from tqdm import tqdm
import numpy as np
import time

_DETAILED = False

num_errors_per_job = []
num_shots_per_job = []

num_errors_in_first_layout = []
num_shots_in_first_layout = []

for job_id in tqdm(md['job_id']):
    num_shots_per_job_ = 0
    job = provider.retrieve_job(job_id)
    layouts = job.initial_layouts()

    num_errors_per_layout = []
    for idx, layout in tqdm(enumerate(list(layouts))):
        
        # Get the layout + noise
        layout_des = job.deserialize_layout(layout)    
        layout = list(layout_des['link_qubit'].values()) + list(layout_des['code_qubit'].values())
        qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=ROUNDS)
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability = 0, #if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)

        # Decode the counts
        memory = job.result().get_memory(idx)
        result_kde = cpp_soft_info.decode_IQ_kde(model, memory, ROUNDS, int(LOGICAL), _RESETS, 
                                                qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                                nb_intervals=-1)
        
        num_errors_per_layout.append(result_kde.num_errors)
        num_shots_per_job_ += memory.shape[0]
        if idx == 0:
            num_shots_in_first_layout.append(memory.shape[0])
    num_shots_per_job.append(num_shots_per_job_)
    num_errors_per_job.append(np.sum(num_errors_per_layout))

    num_errors_in_first_layout.append(num_errors_per_layout[0])

    print(f"Job {job_id} had {num_errors_per_layout} errors per layout out of total {num_shots_per_job} shots")




90it [00:20,  4.31it/s]00:00<?, ?it/s]
 10%|█         | 1/10 [00:21<03:13, 21.46s/it]

Job cqqf0d9v78p0008mdt3g had [1, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 5, 0, 6, 0, 3, 1, 8, 5, 4, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 4, 0, 3, 1, 2, 0, 2, 1, 4, 0, 6, 0, 7, 1, 4, 0, 4, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 2, 0, 1] errors per layout out of total [9990] shots


90it [00:20,  4.30it/s]
 20%|██        | 2/10 [00:43<02:52, 21.52s/it]

Job cqqf0bsr43fg008x5td0 had [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 5, 0, 1, 1, 2, 1, 4, 3, 2, 7, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 5, 0, 2, 0, 6, 1, 4, 1, 2, 2, 4, 1, 4, 1, 1, 0, 0, 2, 1, 1, 1, 2, 0, 0, 0, 0, 0, 1] errors per layout out of total [9990, 9990] shots


90it [00:20,  4.29it/s]
 30%|███       | 3/10 [01:04<02:30, 21.48s/it]

...

Job cqqf018v78p0008mdt20 had [0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 2, 2, 5, 3, 4, 1, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 2, 1, 4, 1, 5, 0, 1, 0, 3, 1, 4, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0] errors per layout out of total [9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


90it [00:20,  4.43it/s]
 90%|█████████ | 9/10 [03:15<00:21, 21.59s/it]

Job cqqezzq417q0008ytz8g had [0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 1, 5, 3, 14, 2, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 4, 0, 3, 1, 5, 0, 5, 0, 4, 0, 5, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0] errors per layout out of total [9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


90it [00:20,  4.45it/s]
100%|██████████| 10/10 [03:36<00:00, 21.68s/it]

Job cqqezy7r43fg008x5tc0 had [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 3, 0, 2, 0, 3, 0, 5, 0, 3, 2, 8, 2, 5, 0, 2, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 3, 3, 0, 1, 0, 2, 0, 2, 0, 7, 0, 3, 1, 5, 0, 1, 0, 7, 0, 3, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0] errors per layout out of total [9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


In [24]:
# Calculate the error rate
print('num_errors_per_job', num_errors_per_job)
print('num_shots_per_job', num_shots_per_job)
print('sum(num_errors_per_job)', np.sum(num_errors_per_job))
print('sum(num_shots_per_job)', np.sum(num_shots_per_job))

error_rate = np.sum(num_errors_per_job) / np.sum(num_shots_per_job)

print(f"The error rate is {error_rate*100:.2f} %")

num_errors_per_job [97, 89, 81, 83, 57, 92, 77, 65, 88, 91]
num_shots_per_job [9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990]
sum(num_errors_per_job) 820
sum(num_shots_per_job) 99900
The error rate is 0.82 %


In [25]:
# Individual error rates on first layout

print('num_errors_in_first_layout', np.sum(num_errors_in_first_layout))
print('num_shots_in_first_layout', np.sum(num_shots_in_first_layout))

error_rate_first_layout = np.sum(num_errors_in_first_layout) / np.sum(num_shots_in_first_layout)

print(f"The error rate on the first layout is {error_rate_first_layout*100:.2f} %")

num_errors_in_first_layout 2
num_shots_in_first_layout 1110
The error rate on the first layout is 0.18 %
